<a href="https://colab.research.google.com/github/pyoph/ProjetHDP/blob/main/projetHDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cvxpy as cp
import numpy as np

In [11]:
#Premiers essais relaxation convexe (p,q) SBM : X_i,i = 1


np.random.seed(1)

#Paramètres
n = 3
#Définitions des seuils
p = np.random.uniform(0,1)
q = np.random.uniform(0,1)

J3 = np.array([[1,1,1],[1,1,1],[1,1,1]])

#Tuning parameter
l = (np.log(p/(1 - p)) - np.log(q/(1 - q)))/(np.log(p) - np.log(q) + np.log(1 - q) - np.log(1 - p))

#Définition de la matrice Z à estimer

Z = np.random.choice([0, 1], size=(n, n))
Z[0][0] = 1
Z[1][1] = 1
Z[2][2] = 1


print(Z)

C = np.random.uniform(0,1,size=(n,n))

#Définition de la matrice C observée

for i in range(3):
  for j in range(3) :
    if (Z[i][j] == 0) : C[i][j] = np.random.choice([0,1],p = [1 -p,p])
    else : C[i][j] = np.random.choice([0,1],p = [1 -q,q])



print(C)

A = []
b = []
for i in range(n):
    b.append(1)

    A.append(np.array([[1,0,0], [0,0,0], [0,0,0]]))
    A.append(np.array([[0,0,0], [0,1,0], [0,0,0]]))
    A.append(np.array([[0,0,0], [0,0,0], [0,0,1]]))



# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(n)
]
prob = cp.Problem(cp.Minimize(cp.trace((C - l*J3) @ X)),
                  constraints)
prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

[[1 1 1]
 [1 1 0]
 [0 1 1]]
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 0. 1.]]
The optimal value is -0.999999929688672
A solution X is
[[1.         0.         0.        ]
 [0.         1.         0.99999993]
 [0.         0.99999993 1.        ]]
